In [1]:
#All these packages need to be installed from pip
#For ML
import sklearn
import sklearn.feature_extraction.text
import sklearn.decomposition
from sklearn import preprocessing, linear_model
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.datasets import fetch_20newsgroups, make_blobs
from sklearn.feature_extraction.text import TfidfVectorizer  #Feature extraction
from sklearn.naive_bayes import MultinomialNB #Our learner.
from sklearn.pipeline import make_pipeline
from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestRegressor
from sklearn.neural_network import MLPClassifier
from sklearn import neighbors
import pandas as pd

import nltk #For tokenizing and normalizing
import numpy as np #arrays
import matplotlib.pyplot as plt #Plots
from matplotlib.colors import ListedColormap
import seaborn #Makes plots look nice, also heatmaps
import scipy as sp #for interp

%matplotlib inline

#These are from the standard library
import collections
import os
import os.path
import random
import re
import glob
import pandas
import requests
import json
import math

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [9]:
df = pandas.read_pickle('com_data_test.pkl')
#df = df.sample(frac = .1)

#splitting data
data_train, data_test = train_test_split(df, test_size=0.3, random_state=123)
data_train['is_train'] = True
data_test['is_train'] = False

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [10]:
df[:3]

,sub_text_x,com_text,com_delta_received,com_delta_from_op,com_upvotes,tokenized_com,normalized_com,tokenized_sub,normalized_sub,tuple,KL,JS,sub_text_y,reduced_tokens,kmeans,kmeans_inter
com_id,,,,,,,,,,,,,,,,
cas07fi,Here is why. The Emperor had an extraordinary ...,"I'm not familiar with the EU, but based on you...",False,False,3,"[I, 'm, not, familiar, with, the, EU, ,, but, ...","[familiar, eu, base, descript, end, justifi, m...","[Here, is, why, ., The, Emperor, had, an, extr...","[whi, emperor, extraordinari, foresight, attac...",(Here is why. The Emperor had an extraordinary...,0.493464,0.110792,NaN,NaN,NaN,NaN
d3uo9sy,"To start, I'm probably going to mostly use my ...",The vast majority of redditors on /r/$TOPIC ar...,False,False,1,"[The, vast, majority, of, redditors, on, /r/, ...","[vast, major, redditor, topic, complet, unqual...","[To, start, ,, I, 'm, probably, going, to, mos...","[start, probabl, go, use, comment, becaus, eas...","(To start, I'm probably going to mostly use my...",0.387678,0.093431,NaN,NaN,NaN,NaN
d6ab2b8,"First up, I'm not suggesting any kind of legal...",Men's macho behavior escalates in male-only pl...,False,False,1,"[Men, 's, macho, behavior, escalates, in, male...","[men, macho, behavior, escal, place, prison, n...","[First, up, ,, I, 'm, not, suggesting, any, ki...","[first, suggest, ani, kind, legal, otherwis, t...","(First up, I'm not suggesting any kind of lega...",0.141931,0.038501,NaN,NaN,NaN,NaN


# Idea 1: try classification for all posts

## Logistic regression

In [4]:
#turn the training dataset into a tf-idf matrix
TFVectorizer = sklearn.feature_extraction.text.TfidfVectorizer(max_df=100, min_df=2, ngram_range=(1, 10),stop_words='english', norm='l2')
TFVects = TFVectorizer.fit_transform(data_train['com_text'])
TFVects.shape #(2055, 5802)
#print(TFVects.shape)


(2014, 15995)

In [6]:
#combining tfidf features with liguistic features and clustering labels
tfdf = pd.DataFrame(TFVects.toarray())
features_train = pd.concat([tfdf, data_train[['com_upvotes', 'KL', 'JS', 'kmeans']]], axis = 1, ignore_index = False)
features_test = pd.concat([tfdf, data_test[['com_upvotes', 'KL', 'JS', 'kmeans']]], axis = 1, ignore_index = False)
features_train[:3]

,0,1,2,3,4,5,6,7,8,9,...,15985,15986,15987,15988,15989,15990,15991,15992,15993,15994
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
#Perform PCA on this matrix to pick significant features
#perform pca
PCA = sklearn.decomposition.PCA
pca = PCA().fit(features_train)
reduced_data = pca.transform(features_train)


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [14]:
#use scree plot to determine the number of dimensions
n = TFVects.shape[0]
fig = plt.figure(figsize=(12,5))
ax1 = fig.add_subplot(121)
eigen_vals = np.arange(n) + 1
ax1.plot(eigen_vals, pca.explained_variance_ratio_, 'ro-', linewidth=2)
ax1.set_title('Scree Plot')
ax1.set_xlabel('Principal Component')
ax1.set_ylabel('Proportion of Explained Variance')

ax2 = fig.add_subplot(122)
eigen_vals = np.arange(20) + 1
ax2.plot(eigen_vals, pca.explained_variance_ratio_[:20], 'ro-', linewidth=2)
ax2.set_title('Scree Plot (First 20 Principal Components)')
ax2.set_xlabel('Principal Component')
ax2.set_ylabel('Proportion of Explained Variance')
plt.show()

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/pandas/indexes/range.py:432: RuntimeWarning: unorderable types: int() < str(), sort order is undefined for incomparable objects
  return self._int64index.union(other)


In [ ]:
#perform pca
PCA = sklearn.decomposition.PCA
pca = PCA().fit(features_train)
reduced_data = pca.transform(TFVects.toarray())

#turn the test dataset into a tf-idf
TFVects_test = TFVectorizer.transform(data_test['com_text'])
#print(TFVects_test.shape)
reduced_data_test = pca.transform(TFVects_test.toarray())



In [ ]:
#PCA was not able to separate delta-receiving comments and others at all.
#use scree plot to determine the number of dimensions
n = TFVects.shape[0]
fig = plt.figure(figsize=(12,5))
ax1 = fig.add_subplot(121)
eigen_vals = np.arange(n) + 1
ax1.plot(eigen_vals, pca.explained_variance_ratio_, 'ro-', linewidth=2)
ax1.set_title('Scree Plot')
ax1.set_xlabel('Principal Component')
ax1.set_ylabel('Proportion of Explained Variance')

ax2 = fig.add_subplot(122)
eigen_vals = np.arange(20) + 1
ax2.plot(eigen_vals, pca.explained_variance_ratio_[:20], 'ro-', linewidth=2)
ax2.set_title('Scree Plot (First 20 Principal Components)')
ax2.set_xlabel('Principal Component')
ax2.set_ylabel('Proportion of Explained Variance')
plt.show()

In [ ]:
#predicting with first ten
X = reduced_data[:, :10]
Y = np.array([int(label) for label in data_train['com_delta_received']]) #Transform our predictor variable. 
              
#fitting logistic regresion
logistic = linear_model.LogisticRegression()
logistic.fit(X, Y)
print("This logistic model using top ten components fits {} of our training set".format(logistic.score(X,Y)))

X_test = reduced_data_test[:, :10]
Y_test = np.array([int(label) for label in data_test['com_delta_received']])
print("This logistic model using top ten components fits {} of our testing set".format(logistic.score(X_test, Y_test)))

# Idea 2: try classification sparately for each "cluster"

In [ ]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml
